Example Queries for Cursor-Based Pagination

In [ ]:
from phoenix.db import models
from sqlalchemy import and_, create_engine, select
from sqlalchemy.orm import aliased, sessionmaker

In [ ]:
PostgresSession = sessionmaker(
    create_engine(
        "postgresql+psycopg://localhost:5432/postgres?user=postgres&password=mysecretpassword",
        echo=True,
    ),
    expire_on_commit=False,
)
SqliteSession = sessionmaker(
    create_engine("sqlite:////Users/xandersong/.phoenix/phoenix.db", echo=True),
    expire_on_commit=False,
)

- filter: None
- sort: None
- after: None
- before: None
- first: 10
- last: None

In [ ]:
page_size = 10
with SqliteSession() as session:
    span_ids = session.scalars(select(models.Span.id).limit(page_size)).all()
span_ids

- filter: None
- sort: None
- after: 5
- before: None
- first: 10
- last: None

In [ ]:
cursor = 10
page_size = 10
with SqliteSession() as session:
    span_ids = session.scalars(
        select(models.Span.id).where(models.Span.id >= cursor).limit(page_size)
    ).all()
span_ids

- filter: """span_kind == 'LLM'"""
- sort: None
- after: 10
- before: None
- first: 10
- last: None

In [ ]:
cursor = 10
page_size = 10
with SqliteSession() as session:
    span_ids = session.scalars(
        select(models.Span.id)
        .where(models.Span.span_kind == "LLM")
        .where(models.Span.id >= cursor)
        .limit(page_size)
    ).all()
span_ids

- filter: None
- sort: prompt token count
- after: 10
- before: None
- first: 10
- last: None

In [ ]:
cursor = 100
page_size = 10
with SqliteSession() as session:
    prompt_tokens = models.Span.attributes[["llm", "token_count", "prompt"]]
    for index, span in enumerate(
        session.execute(
            select(models.Span.id, prompt_tokens)
            .where(models.Span.id >= cursor)
            .order_by(prompt_tokens, models.Span.id)
            .limit(page_size)
        )
    ):
        print(f"{index=} {span=}")

- filter: attributes["llm.prompt.tokens] < 50
- sort: evals["Q&A Correctness"].score DESC
- after: 10
- before: None
- first: 10
- last: None

In [ ]:
span

In [ ]:
span.attributes["llm"]

In [ ]:
cursor = 100
page_size = 10
A = aliased(models.SpanAnnotation, name="A")
with SqliteSession() as session:
    for index, (span, score) in enumerate(
        session.execute(
            select(models.Span, A.score)
            .join(
                A,
                onclause=and_(
                    A.span_rowid == models.Span.id,
                    A.name == "Q&A Correctness",
                    A.annotator_kind == "LLM",
                ),
            )
            .where(models.Span.id >= cursor)
            .order_by(A.score.desc(), models.Span.id)
        )
    ):
        print(f"{index=} {span.id=} {score=}")